# Matlab script for MTEX pole figures
Make sure to switch to the Matlab jupyter server before running this script.  
It might be necessary to download MTEX locally to run `startup.m`.

In [ ]:
run('/mxn/home/danmax-user/scripts/mtex-5p10p0/startup.m')

In [ ]:
%% Specify Crystal and Specimen Symmetries

% crystal symmetry
CS = crystalSymmetry('m-3m', [4.1 4.1 4.1], 'mineral', 'Silver');

% specimen symmetry
SS = specimenSymmetry('1');

% DanMAX plotting convention
setMTEXpref('xAxisDirection','west');
setMTEXpref('zAxisDirection','intoPlane');

#### Find file names and Miller indices

In [ ]:
% path to files
pname = '/data/visitors/danmax/PROPOSAL/VISIT/';
pname = '/data/visitors/danmax/20230160/2023101208/scripts_revised/texture/';
% find partialPF files
files = dir(strcat(pname,'*partialPF*.txt'));
fname = append({files(:).folder},'/' ,{files(:).name});

h={};
for i = 1:length(fname)
    hkl = split(fname{i},'_');
    hkl = split(hkl(end),'.');
    hkl=hkl(1);
    if strlength(hkl) ~= 3;
        display(strcat('ambiguous hkl for',fname{i}))
        display(sprintf('Please manually add "h{%d} = Miller(h,k,l,CS)"',i))
        h = [h ;  hkl];
    end
    h = [h ; {Miller(str2num(hkl{1}(1)),str2num(hkl{1}(2)),str2num(hkl{1}(3)),CS)}];
end

% h{i} = Miller(h,k,l,CS)

In [ ]:
%% Import the Data

% create a Pole Figure variable containing the data
pf = PoleFigure.load(fname,h,CS,SS,'interface','generic',...
  'ColumnNames', { 'Polar Angle' 'Azimuth Angle' 'Intensity'});

In [ ]:
%% Correct Data

rot = rotation.byEuler(180*degree,0*degree,0*degree);
pf = rotate(pf,rot);

In [ ]:
%% Estimate ODF
odf = calcODF(pf,'silent')
calcError(odf,pf)

#### Plot experimental pole figures

In [ ]:
%% Plot reconstructed pole figures
figure;
set(gcf,'PaperPositionMode','auto');
plot(pf);
%mtexColorbar;
mtexColorMap parula;

#### Plot reconstructed pole figures

In [ ]:
figure;
set(gcf,'PaperPositionMode','auto');
plotPDF(odf,pf.h);
%mtexColorbar;
mtexColorMap parula;

#### Plot absolute difference pole figures

In [ ]:
figure;
set(gcf,'PaperPositionMode','auto');
plotDiff(pf,odf);
mtexColorbar;
mtexColorMap parula;

#### Plot inverse pole figures

In [ ]:
figure;
plotIPDF(odf,[xvector,yvector,zvector]);
mtexColorbar;
mtexColorMap parula;